In [1]:
#@title Import libraries
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

from easydict import EasyDict
import collections
import json
import yaml
import os.path as osp
from pprint import pprint

import numpy as np
import torch
import clip
import tensorflow.compat.v1 as tf # version .1
from tqdm import tqdm
from scipy.special import softmax

from matplotlib import pyplot as plt
from matplotlib import patches
import cv2
from PIL import Image
import PIL.ImageColor as ImageColor
import PIL.ImageDraw as ImageDraw
import PIL.ImageFont as ImageFont


#@title Define hyperparameters
FLAGS = {
    'prompt_engineering': True,
    'this_is': True,
    
    'temperature': 100.0,
    'use_softmax': False,
}
FLAGS = EasyDict(FLAGS)

# Global matplotlib settings
SMALL_SIZE = 16#10
MEDIUM_SIZE = 18#12
BIGGER_SIZE = 20#14
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# Parameters for drawing figure.
display_input_size = (10, 10)
overall_fig_size = (18, 24)
line_thickness = 2
fig_size_w = 35
# fig_size_h = min(max(5, int(len(category_names) / 2.5) ), 10)
mask_color =   'red'
alpha = 0.5







def article(name):  #% A/An
  return 'an' if name[0] in 'aeiou' else 'a'

def processed_name(name, rm_dot=False):  #% '_' for lvis, '/' for obj365
  res = name.replace('_', ' ').replace('/', ' or ').lower()
  if rm_dot:
    res = res.rstrip('.')
  return res


# //////////////////////////////////////////////////////////////
single_template = [  #% Only one
    'a photo of {article} {}.'
]
# //////////////////////////////////////////////////////////////
multiple_templates = [
    'There is {article} {} in the scene.',
    'There is the {} in the scene.',
    'a photo of {article} {} in the scene.',
    'a photo of the {} in the scene.',
    'a photo of one {} in the scene.',

    'itap of {article} {}.',
    'itap of my {}.',  # itap: I took a picture of
    'itap of the {}.',
    
    'a photo of {article} {}.',
    'a photo of my {}.',
    'a photo of the {}.',
    'a photo of one {}.',
    'a photo of many {}.',

    'a good photo of {article} {}.',
    'a good photo of the {}.',
    'a bad photo of {article} {}.',
    'a bad photo of the {}.',
    'a photo of a nice {}.',
    'a photo of the nice {}.',
    'a photo of a cool {}.',
    'a photo of the cool {}.',
    'a photo of a weird {}.',
    'a photo of the weird {}.',

    'a photo of a small {}.',
    'a photo of the small {}.',
    'a photo of a large {}.',
    'a photo of the large {}.',

    'a photo of a clean {}.',
    'a photo of the clean {}.',
    'a photo of a dirty {}.',
    'a photo of the dirty {}.',

    'a bright photo of {article} {}.',
    'a bright photo of the {}.',
    'a dark photo of {article} {}.',
    'a dark photo of the {}.',

    'a photo of a hard to see {}.',
    'a photo of the hard to see {}.',
    'a low resolution photo of {article} {}.',
    'a low resolution photo of the {}.',
    'a cropped photo of {article} {}.',
    'a cropped photo of the {}.',
    'a close-up photo of {article} {}.',
    'a close-up photo of the {}.',
    'a jpeg corrupted photo of {article} {}.',
    'a jpeg corrupted photo of the {}.',
    'a blurry photo of {article} {}.',
    'a blurry photo of the {}.',
    'a pixelated photo of {article} {}.',
    'a pixelated photo of the {}.',

    'a black and white photo of the {}.',
    'a black and white photo of {article} {}.',

    'a plastic {}.',
    'the plastic {}.',

    'a toy {}.',
    'the toy {}.',
    'a plushie {}.',
    'the plushie {}.',
    'a cartoon {}.',
    'the cartoon {}.',

    'an embroidered {}.',
    'the embroidered {}.',

    'a painting of the {}.',
    'a painting of a {}.',
]
# //////////////////////////////////////////////////////////////




clip.available_models()
model, preprocess = clip.load("ViT-B/32")




#///////////////////////////////////////////
#     Build text embedding function:
def build_text_embedding(categories):
  if FLAGS.prompt_engineering:  ## prompt engineering => some templates
    templates = multiple_templates
  else:
    templates = single_template

  with torch.no_grad():
    all_text_embeddings = []  ## All Text Embeddings
    print('Building text embeddings...')
    
    for category in tqdm(categories):
      texts = [
        template.format(processed_name(category['name'], rm_dot=True),
                        article=article(category['name']))
        for template in templates
      ]
      if FLAGS.this_is:
        texts = [
                 'This is ' + text if text.startswith('a') or text.startswith('the') else text 
                 for text in texts
                 ]
      texts = clip.tokenize(texts) #@ Returns a LongTensor containing tokenized sequences of given text input(s).
      #TODO print("texts[tokenized sequences of given text inputs]: ", texts) ## This can be used as the input to the model.
      if torch.cuda.is_available():
        texts = texts.cuda()
      
      #@ Given a batch of text tokens, returns the text features encoded by the language portion of the CLIP model.  
      text_embeddings = model.encode_text(texts) #embed with text encoder 
      text_embeddings /= text_embeddings.norm(dim=-1, keepdim=True)
      text_embedding = text_embeddings.mean(dim=0)
      text_embedding /= text_embedding.norm()
      all_text_embeddings.append(text_embedding)
    ## FOR ENDING  
    all_text_embeddings = torch.stack(all_text_embeddings, dim=1)
    if torch.cuda.is_available():
      all_text_embeddings = all_text_embeddings.cuda()
  ## WITH ENDING  
  return all_text_embeddings.cpu().numpy().T




## Helper Function ##
numbered_categories = [{'name': str(idx), 'id': idx,} for idx in range(50)]
# print(numbered_categories)  # [{'name': '0', 'id': 0}, {, ..., {'name': '49', 'id': 49}]
numbered_category_indices = {cat['id']: cat for cat in numbered_categories}
# print(numbered_category_indices)  # 0: {'name': '0', 'id': 0}, ..., 49: {'name': '49', 'id': 49}}

#@ Non Maximum Suppression
#@ dets: [N, 4] 
#@ scores: [N,]
#@ thresh[Float]: iou threshold.
#@ max_dets[Int]
def nms(dets, scores, thresh, max_dets=1000):
  y1 = dets[:, 0]
  x1 = dets[:, 1]
  y2 = dets[:, 2]
  x2 = dets[:, 3]

  areas = (x2 - x1) * (y2 - y1)
  order = scores.argsort()[::-1]

  keep = []
  while order.size > 0 and len(keep) < max_dets:
    i = order[0]
    keep.append(i)

    xx1 = np.maximum(x1[i], x1[order[1:]])
    yy1 = np.maximum(y1[i], y1[order[1:]])
    xx2 = np.minimum(x2[i], x2[order[1:]])
    yy2 = np.minimum(y2[i], y2[order[1:]])

    w = np.maximum(0.0, xx2 - xx1)
    h = np.maximum(0.0, yy2 - yy1)
    intersection = w * h
    overlap = intersection / (areas[i] + areas[order[1:]] - intersection + 1e-12)

    inds = np.where(overlap <= thresh)[0]
    order = order[inds + 1]
  ## WHILE ENDING 
  return keep


## Graph and Session Init ##
session = tf.Session(graph=tf.Graph())
saved_model_dir = './image_path_v2' #@param {type:"string"}
_ = tf.saved_model.loader.load(session, ['serve'], saved_model_dir) # Restoring parameters from ./image_path_v2/variables/variables








def paste_instance_masks(masks, detected_boxes, image_height, image_width):
  def expand_boxes(boxes, scale):
    """Expands an array of boxes by a given scale."""
    w_half = boxes[:, 2] * .5
    h_half = boxes[:, 3] * .5
    x_c = boxes[:, 0] + w_half
    y_c = boxes[:, 1] + h_half

    w_half *= scale
    h_half *= scale

    boxes_exp = np.zeros(boxes.shape)
    boxes_exp[:, 0] = x_c - w_half
    boxes_exp[:, 2] = x_c + w_half
    boxes_exp[:, 1] = y_c - h_half
    boxes_exp[:, 3] = y_c + h_half

    return boxes_exp
  
  _, mask_height, mask_width = masks.shape
  scale = max((mask_width + 2.0) / mask_width, (mask_height + 2.0) / mask_height)

  ref_boxes = expand_boxes(detected_boxes, scale)  ## Expand the boxes
  ref_boxes = ref_boxes.astype(np.int32)
  padded_mask = np.zeros((mask_height + 2, mask_width + 2), dtype=np.float32)
  segms = []
  for mask_ind, mask in enumerate(masks):
    im_mask = np.zeros((image_height, image_width), dtype=np.uint8)
    padded_mask[1:-1, 1:-1] = mask[:, :]  # Process mask inside bounding boxes.

    ref_box = ref_boxes[mask_ind, :]
    w = ref_box[2] - ref_box[0] + 1
    h = ref_box[3] - ref_box[1] + 1
    w = np.maximum(w, 1)
    h = np.maximum(h, 1)

    mask = cv2.resize(padded_mask, (w, h))
    mask = np.array(mask > 0.5, dtype=np.uint8)

    x_0 = min(max(ref_box[0], 0), image_width)
    x_1 = min(max(ref_box[2] + 1, 0), image_width)
    y_0 = min(max(ref_box[1], 0), image_height)
    y_1 = min(max(ref_box[3] + 1, 0), image_height)

    im_mask[y_0:y_1, x_0:x_1] = mask[
        (y_0 - ref_box[1]):(y_1 - ref_box[1]),
        (x_0 - ref_box[0]):(x_1 - ref_box[0])
    ]
    segms.append(im_mask)

  segms = np.array(segms)
  assert masks.shape[0] == segms.shape[0]
  
  return segms   #@ Return numpy array [N, image_height, image_width] => the instance masks pasted on the image canvas.













#@title Visualization
STANDARD_COLORS = ['AliceBlue', 'Chartreuse', 'Aqua', 'BlueViolet', 'DarkOrange', 'DeepPink', 'DeepSkyBlue', 'GhostWhite', 'Gold', 'LightBlue', 'LightCoral', 'LightSalmon', 'LightSeaGreen', 'LightSkyBlue', 'LightSlateGray', 'LightSlateGrey', 'LightSteelBlue', 'Olive', 'Orange', 'Purple', 'RosyBrown', 'RoyalBlue', 'SaddleBrown', 'Green', 'SandyBrown', 'SeaGreen', 'SeaShell', 'Sienna', 'Silver','Turquoise', 'Violet', 'Wheat', 'White', 'WhiteSmoke', 'Yellow', 'YellowGreen']
  
#* #########################################################
#@ Draw_boxes function
#@ image: a PIL.Image object.
#@ display_str_list: list of strings to display in box (out of range => displaying below the box)
#@ use_normalized_coordinates: True (default) => treat coordinates as relative to the image. Otherwise => absolute.
def draw_bounding_box_on_image(image, ymin, xmin, ymax, xmax, 
                               color='red', thickness=4,
                               display_str_list=(),
                               use_normalized_coordinates=True,user=0,):
  if user==2 or user==3:
    return
  
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  if use_normalized_coordinates:
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
  else:
    (left, right, top, bottom) = (xmin, xmax, ymin, ymax)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top), (left, top)], width=thickness, fill=color)
  
  try:
    font = ImageFont.truetype('arial.ttf', 24)
  except IOError:
    font = ImageFont.load_default()

  #@ Displayed string position adjustment 
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)   # display_str has top and bottom margin_(0.05x).
  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = bottom + total_display_str_height
    
  for display_str in display_str_list[::-1]:  
    text_left = min(5, left)
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin), (left + text_width, text_bottom)], fill=color)
    draw.rectangle([(left + text_width, text_bottom - text_height - 2 * margin), (right + margin, text_bottom)], fill='LightGreen')
    
    draw.text((left + margin, text_bottom - text_height - margin),display_str, fill='black', font=font)
    #draw.text((left + margin + text_width*2, text_bottom - text_height - margin), display_str, fill='black', font=font)
    text_bottom -= text_height - 2 * margin



#* #########################################################
#@ Draw_box on image (numpy array form) {CALL draw_bounding_box_on_image()}
#@ image: a numpy array with shape [height, width, 3].
def draw_bounding_box_on_image_array(image, ymin, xmin, ymax, xmax, user=0,
                                     color='red', thickness=4,
                                     display_str_list=(),
                                     use_normalized_coordinates=True):
  image_pil = Image.fromarray(np.uint8(image)).convert('RGB')
  #print(user)
  draw_bounding_box_on_image(image_pil, ymin, xmin, ymax, xmax, color, thickness, display_str_list, use_normalized_coordinates, user=user)
  np.copyto(image, np.array(image_pil))  # image flush





def draw_mask_on_image_array(image, mask, color='red', alpha=0.4):
  if image.dtype != np.uint8:
    raise ValueError('`image` not of type np.uint8')
  if mask.dtype != np.uint8:
    raise ValueError('`mask` not of type np.uint8')
  if np.any(np.logical_and(mask != 1, mask != 0)):
    raise ValueError('`mask` elements should be in [0, 1]')
  if image.shape[:2] != mask.shape:
    raise ValueError('The image has spatial dimensions %s, but the mask has dimensions %s' % (image.shape[:2], mask.shape))
  rgb = ImageColor.getrgb(color)
  pil_image = Image.fromarray(image)
  solid_color = np.expand_dims(np.ones_like(mask), axis=2) * np.reshape(list(rgb), [1, 1, 3])
  pil_solid_color = Image.fromarray(np.uint8(solid_color)).convert('RGBA')
  pil_mask = Image.fromarray(np.uint8(255.0*alpha*mask)).convert('L')
  pil_image = Image.composite(pil_solid_color, pil_image, pil_mask)
  np.copyto(image, np.array(pil_image.convert('RGB')))
  
def plot_mask(color, alpha, original_image, mask):  #! Simialr to draw_mask_on_image_array()
  rgb = ImageColor.getrgb(color)
  pil_image = Image.fromarray(original_image)

  solid_color = np.expand_dims(np.ones_like(mask), axis=2) * np.reshape(list(rgb), [1, 1, 3])
  pil_solid_color = Image.fromarray(np.uint8(solid_color)).convert('RGBA')
  pil_mask = Image.fromarray(np.uint8(255.0*alpha*mask)).convert('L')
  pil_image = Image.composite(pil_solid_color, pil_image, pil_mask)
  ## Maintain the original image
  img_w_mask = np.array(pil_image.convert('RGB'))
  return img_w_mask



#* #########################################################
#@ Visualize labeled boxes, scores
#@ image: uint8 numpy array [img_height, img_width, 3]
#@ boxes: numpy array [N, 4]
#@ classes: numpy array [N]. Note that class indices are 1-based and match the keys in the label map.
#@ scores: numpy array [N] or None(assumes plotted boxes are groundtruth boxes. Plot all boxes as black without classes/scores)
#@ category_index: a dict [[category index `id`, category name `name`] keyed by category indices]
global scores_all
global indices_fg
def visualize_boxes_and_labels_on_image_array(image, boxes, classes, scores, category_index,user,
                                              instance_masks=None, instance_boundaries=None,
                                              use_normalized_coordinates=False,
                                              max_boxes_to_draw=20, min_score_thresh=.5, #max number of boxes and  minimum score to visualize
                                              agnostic_mode=False,
                                              line_thickness=4,
                                              groundtruth_box_visualization_color='black',
                                              skip_scores=False, skip_labels=False,
                                              mask_alpha=0.4,
                                              plot_color=None,):
  box_to_display_str_map = collections.defaultdict(list)
  box_to_color_map = collections.defaultdict(str)
  box_to_instance_masks_map = {}
  box_to_score_map = {}
  box_to_instance_boundaries_map = {}
  
  #print(user)
  
  if not max_boxes_to_draw:
    max_boxes_to_draw = boxes.shape[0]

  #//print(scores)
  #//print("boxes.shape: ", boxes.shape[0])
  for i in range(min(max_boxes_to_draw, boxes.shape[0])):
    print(i)
    if scores is None or scores[i] > min_score_thresh:
      box = tuple(boxes[i].tolist())
      
      if instance_masks is not None:
        box_to_instance_masks_map[box] = instance_masks[i]
      if instance_boundaries is not None:
        box_to_instance_boundaries_map[box] = instance_boundaries[i]
      if scores is None:
        box_to_color_map[box] = groundtruth_box_visualization_color
      
      else:
        display_str = ''
        if not skip_labels:
          if not agnostic_mode:
            #print(scores_all[i])
            #print(category_indices[np.argmax(scores_all[i])])
            #print(category_indices)
            #//class_name = category_index[np.argmax(scores_all[classes[i]])]['name']k
            #//print("the number is: ", classes[i], "\n","the possibility is: ", scores_all[classes[i]])
            class_name = category_index[np.argmax(scores_all[indices_fg[i]])]['name']
            #print(list(category_index.keys()))
            #if classes[i] in list(category_index.keys()):
              #print(classes[i])
              #!class_name = category_index[classes[i]]['name']
              #print(class_name)
            #else:
            #  class_name = 'N/A'
            display_str = str(class_name)
            #print(display_str)
            
        ## SKIP_LABELS IF ENDING
        if not skip_scores:
          #if not display_str:
          #  display_str = '{}%'.format(int(100*scores[i]))
          #else:
          #  float_score = ("%.2f" % scores[i]).lstrip('0')
          #  display_str = '{}: {}'.format(display_str, float_score)
          float_score = ("%.2f" % float(np.max(scores_all[indices_fg[i]])*100))
          display_str = '{}: {}%'.format(display_str, float_score)
          box_to_score_map[box] = int(100*scores[i])
          
        ## SKIP_SCORES IF ENDING 
        box_to_display_str_map[box].append(display_str)
        if plot_color is not None:
          box_to_color_map[box] = plot_color
        elif agnostic_mode:
          box_to_color_map[box] = 'DarkOrange'
        else:
          box_to_color_map[box] = STANDARD_COLORS[
              classes[i] % len(STANDARD_COLORS)]
  
  ## Handle the case when box_to_score_map is empty.
  if box_to_score_map:
    box_color_iter = sorted(box_to_color_map.items(), key=lambda kv: box_to_score_map[kv[0]])
  else:
    box_color_iter = box_to_color_map.items()

  ## Draw all boxes onto image.
  ## UI Change parameter: user ##
  for box_color, tmpUser in zip(box_color_iter, user):
    box, color = box_color
    ymin, xmin, ymax, xmax = box
    if instance_masks is not None:
      draw_mask_on_image_array(image, box_to_instance_masks_map[box], color=color, alpha=mask_alpha)
    if instance_boundaries is not None:
      draw_mask_on_image_array(image, box_to_instance_boundaries_map[box], color='red', alpha=1.0)
      
    draw_bounding_box_on_image_array(image, ymin, xmin, ymax, xmax, user=tmpUser,
                                     color=color, thickness=line_thickness,
                                     display_str_list=box_to_display_str_map[box],
                                     use_normalized_coordinates=use_normalized_coordinates)
    
    #print(type(image))
    im = Image.fromarray(image)
    im.save("tmp/ui.jpg")
    #//cv2.imwrite("tmp/ui.png", image)
  return image   #@ Return uint8 numpy array [img_height, img_width, 3] with overlaid boxes.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


2022-07-18 21:38:19.428500: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Restoring parameters from ./image_path_v2/variables/variables


2022-07-18 21:38:22.695542: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


In [30]:
scores_all

array([[0.37559953, 0.32875124],
       [0.2853251 , 0.2450394 ],
       [0.24868415, 0.21461299],
       [0.27040896, 0.22903839],
       [0.25045556, 0.23159668],
       [0.24574862, 0.2140573 ],
       [0.36556888, 0.31986576],
       [0.23596096, 0.23321402],
       [0.26907587, 0.23978895],
       [0.275301  , 0.24211088],
       [0.32001954, 0.2866404 ],
       [0.23780736, 0.2143878 ],
       [0.2884213 , 0.26526436],
       [0.23272376, 0.21258655],
       [0.36639145, 0.3003508 ],
       [0.26842093, 0.22223848],
       [0.25790298, 0.20088194],
       [0.24730152, 0.21247607],
       [0.23920766, 0.20116681],
       [0.23907755, 0.22857589],
       [0.29997778, 0.25150838],
       [0.25656635, 0.22271009],
       [0.25664788, 0.22105701],
       [0.25040177, 0.21729903],
       [0.2563432 , 0.2264043 ],
       [0.31292802, 0.28262672],
       [0.24536589, 0.2183201 ],
       [0.26398167, 0.22470008],
       [0.25014472, 0.22210796],
       [0.30146694, 0.27224702],
       [0.

In [20]:
#@parameter debug
#@indices #array([ 0, 14,  6, 32, 10, 25, 38, 47, 42, 29, 20, 36, 12,  1, 49, 31, 35,
          #9,  3,  8, 15, 44, 30, 27, 39, 16, 22, 21, 24, 41, 48,  4, 23, 28,
          #2, 17, 33,  5, 26, 46, 45, 34, 40, 43, 18, 19, 11,  7, 37, 13])
#@indices_fg #array([40])
#@valid_indices[:max_boxes_to_draw] #array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
                                    #17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                                    #34, 35, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51])

In [2]:
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
import shutil
print("START UI")

class firstWindow(QWidget):
  def __init__(self, *args, **kwargs):
    super(firstWindow, self).__init__(*args, **kwargs)
    self.image_path = './examples/test.jpg'
    self.setWindowTitle('Detect objects. Please input custom labels.')
    self.currentText = ""
    self.bboxMax = 0
    self.category_name_string=""
    
    self.img = cv2.imread(self.image_path)
    self.imageHeight, self.imageWidth, self.bytesPerComponent = self.img.shape
    self.resize(self.imageWidth, self.imageHeight)
    self.move(int((QDesktopWidget().screenGeometry().width()-self.width())/2),
              int((QDesktopWidget().screenGeometry().height()-self.height())/2))  
    
    self.lb_0 = originalLabel(self) 
    self.customLabel = customInputLabel(self)
    self.bboxNumLabel = bboxInputLabel(self)
    self.button_1_Init()
    self.button_2_Init()
    self.button_3_Init()
    self.button_4_Init()
    
  def VisualImage(self):
    #//print(self.image_path)
    self.img = cv2.imread(self.image_path)
    self.imageHeight, self.imageWidth, self.bytesPerComponent = self.img.shape
    self.resize(self.imageWidth, self.imageHeight)
    self.move(int((QDesktopWidget().screenGeometry().width()-self.width())/2),
              int((QDesktopWidget().screenGeometry().height()-self.height())/2)) 
    self.lb_0.setGeometry(QRect(0, 0, self.imageWidth, self.imageHeight))
    bytesPerLine = 3 * self.imageWidth
    cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB, self.img)
    QImg = QImage(self.img.data, self.imageWidth, self.imageHeight, bytesPerLine, QImage.Format_RGB888)
    pixmap = QPixmap.fromImage(QImg)
    self.lb_0.setPixmap(pixmap)
    self.update()
    
  def textchange(self, text):
    self.currentText = text
  def numchange(self, bboxNumbers):
    self.bboxMax = bboxNumbers
    #//print(self.bboxMax)
      
  def showPath(self):
    print(self.image_path)
  
  def button_1_Init(self):
    self.bt = QPushButton(self)
    self.bt.resize(30, 30)
    self.bt.setStyleSheet("background-color:transparent")
    self.bt.setIcon(QIcon("icon/icons8-enter-mac-key-90.png"))
    self.bt.setIconSize(QSize(self.bt.size()))
    self.bt.move(self.customLabel.x()+520, self.lb_0.y()+15)
    self.bt.clicked.connect(self.clickInputCustomLabel)
    
  def clickInputCustomLabel(self):
    self.msg_label = QMessageBox(self)
    self.msg_label.setText("Get custom Labels :  \n" + self.currentText + "\nand Max bbox numbers : \n"+self.bboxMax)
    self.msg_label.setIcon(QMessageBox.Information)
    self.msg_label.show()
    cusLabel = self.currentText.split(',')
    self.category_name_string = ';'.join(cusLabel) ## + background 
    print(self.category_name_string)
       
  def button_2_Init(self):
    self.bt_2 = QPushButton(self)
    self.bt_2.resize(30, 30)
    self.bt_2.setStyleSheet("background-color:transparent")
    self.bt_2.setIcon(QIcon("icon/play.png"))
    self.bt_2.setIconSize(QSize(self.bt_2.size()))
    self.bt_2.move(self.bt.x()+40, self.bt.y())  
    self.bt_2.clicked.connect(self.startDetecting)
  
  ## WITH VILD MODEL ##
  def startDetecting(self):
    print("START DETECTING!")
    global max_boxes_to_draw        ########
    max_boxes_to_draw = int(self.bboxMax) 
    #//print(type(self.bboxMax))
    nms_threshold = 0.6 
    global min_rpn_score_thresh     ########
    min_rpn_score_thresh = 0.9
    min_box_area = 220
    category_names = ['background']+[x.strip() for x in self.category_name_string.split(';')]
    categories = [{'name': item, 'id': idx,} for idx, item in enumerate(category_names)]
    global category_indices         ########
    category_indices = {cat['id']: cat for cat in categories} 
    
    roi_boxes, roi_scores, detection_boxes, scores_unused, box_outputs, detection_masks, visual_features, image_info \
    = session.run(
      ['RoiBoxes:0', 'RoiScores:0', '2ndStageBoxes:0', '2ndStageScoresUnused:0', 'BoxOutputs:0', 'MaskOutputs:0', 'VisualFeatOutputs:0', 'ImageInfo:0'],
      feed_dict={'Placeholder:0': [self.image_path, ]}
    )
    roi_boxes = np.squeeze(roi_boxes, axis=0) 
    roi_scores = np.squeeze(roi_scores, axis=0)
    detection_boxes = np.squeeze(detection_boxes, axis=(0, 2))
    scores_unused = np.squeeze(scores_unused, axis=0)
    box_outputs = np.squeeze(box_outputs, axis=0)
    detection_masks = np.squeeze(detection_masks, axis=0)
    visual_features = np.squeeze(visual_features, axis=0)
    image_info = np.squeeze(image_info, axis=0)
    
    image_scale = np.tile(image_info[2:3, :], (1, 2))
    image_height = int(image_info[0, 0])
    image_width = int(image_info[0, 1])
    global rescaled_detection_boxes ########
    rescaled_detection_boxes = detection_boxes / image_scale
    global image                    ########
    image = np.asarray(Image.open(open(self.image_path, 'rb')).convert("RGB"))
    assert image_height == image.shape[0]
    assert image_width == image.shape[1]
    
    nmsed_indices = nms(detection_boxes, roi_scores, thresh=nms_threshold)
    box_sizes = (rescaled_detection_boxes[:, 2] - rescaled_detection_boxes[:, 0]) * (rescaled_detection_boxes[:, 3] - rescaled_detection_boxes[:, 1])
    global valid_indices            ########
    valid_indices = np.where(
      np.logical_and(
        np.isin(np.arange(len(roi_scores), dtype=np.int64), nmsed_indices),
          np.logical_and(
            np.logical_not(np.all(roi_boxes == 0., axis=-1)),
            np.logical_and(
              roi_scores >= min_rpn_score_thresh,
              box_sizes > min_box_area 
            ))))[0]
    print('Valid Indices Cnt: ', len(valid_indices))
    global detection_roi_scores     ########
    detection_roi_scores = roi_scores[valid_indices][:max_boxes_to_draw, ...]
    detection_boxes = detection_boxes[valid_indices][:max_boxes_to_draw, ...]
    detection_masks = detection_masks[valid_indices][:max_boxes_to_draw, ...]
    detection_visual_feat = visual_features[valid_indices][:max_boxes_to_draw, ...]
    rescaled_detection_boxes = rescaled_detection_boxes[valid_indices][:max_boxes_to_draw, ...]    
    
    global isSaved                  ########
    isSaved = np.zeros(detection_roi_scores.shape)
    
    text_features = build_text_embedding(categories)
    raw_scores = detection_visual_feat.dot(text_features.T)
    global scores_all               ########
    if FLAGS.use_softmax:
      scores_all = softmax(FLAGS.temperature * raw_scores, axis=-1)
    else:
      scores_all = raw_scores
    #//print(scores_all.shape)
    #* RANK
    global indices                  ########
    indices = np.argsort(-np.max(scores_all, axis=1))
    global indices_fg               ########
    indices_fg = np.array([i for i in indices if np.argmax(scores_all[i]) != 0])  
    global indices_fg_2
    indices_fg_2 = np.intersect1d(indices_fg, valid_indices)
    
    ymin, xmin, ymax, xmax = np.split(rescaled_detection_boxes, 4, axis=-1)
    processed_boxes = np.concatenate([xmin, ymin, xmax - xmin, ymax - ymin], axis=-1)
    global segmentations            ########
    segmentations = paste_instance_masks(detection_masks, processed_boxes, image_height, image_width) 
    
    if len(indices_fg) == 0:
      self.mgb_no = QMessageBox(self)
      self.mgb_no.setIcon(QMessageBox.Information)
      self.mgb_no.setText("ViLD does not detect anything belong to the given category")
      self.mgb_no.show()
      print('ViLD does not detect anything belong to the given category')
    else:
      image_with_detections = visualize_boxes_and_labels_on_image_array( \
        np.array(image),
        rescaled_detection_boxes[indices_fg],
        valid_indices[:max_boxes_to_draw][indices_fg],
        detection_roi_scores[indices_fg],   
        category_indices, #numbered_category_indices,
        user=isSaved[indices_fg], 
        instance_masks=segmentations[indices_fg],
        use_normalized_coordinates=False,
        max_boxes_to_draw=max_boxes_to_draw, min_score_thresh=min_rpn_score_thresh,
        skip_scores=False, skip_labels=False,
      )
      #//plt.imshow(image_with_detections)  
      
      print("Update Detection Window")
      win_2 = resultWindow(self)
      win_2.show()
    
  def button_3_Init(self):   
    self.bt_3 = QPushButton(self)
    self.bt_3.resize(25, 25)
    self.bt_3.setIcon(QIcon("icon/icons8-image-document-64"))
    self.bt_3.setStyleSheet("background-color:transparent")
    self.bt_3.setIconSize(QSize(self.bt_3.size()))
    self.bt_3.move(self.bt_2.x()+40, self.bt.y())
    self.bt_3.clicked.connect(self.clickChangeImage)
  
  def clickChangeImage(self):
    #//print("Clicked!")
    QFDialog = QFileDialog()
    image_choose, _ = QFDialog.getOpenFileName(self, 
        "Choose a new image to detect.",
        os.getcwd() + "/examples",
        "All Files (*);;Image Files (*.jpg)")
    if image_choose == "":
      print("NO IMAGE CHOSEN!")
      return
    print("New Iamge: ", image_choose, "\n")
    image_choose = image_choose.split('/')[-1]
    self.image_path = './examples/'+ image_choose
    #//self.showPath() 
    self.VisualImage()
     
  def button_4_Init(self):   
    self.bt_4 = QPushButton(self)
    self.bt_4.resize(25, 25)
    self.bt_4.setIcon(QIcon("icon/icons8-save-50.png"))
    self.bt_4.setStyleSheet("background-color:transparent")
    self.bt_4.setIconSize(QSize(self.bt_4.size()))
    self.bt_4.move(self.bt_3.x()+40, self.bt.y())
    self.bt_4.clicked.connect(self.saveDetectedRes) 
  
  def saveDetectedRes(self):
    shutil.rmtree("./result")
    os.mkdir('result')
    cnt = 0
    global image                    ########
    global rescaled_detection_boxes ########
    raw_image = np.array(image)
    n_boxes = rescaled_detection_boxes.shape[0]
    global indices                  ########
    global detection_roi_scores     ########
    global scores_all               ########
    global isSaved                  ########
    for anno_idx in indices[0:int(n_boxes)]:
      print(rescaled_detection_boxes)
      rpn_score = detection_roi_scores[anno_idx]
      bbox = rescaled_detection_boxes[anno_idx]  ## Get BBox
      scores = scores_all[anno_idx]
      showOrNot = isSaved[anno_idx]
      
      if np.argmax(scores) == 0:
        continue
      if showOrNot==2 or showOrNot==3: # if showOrNot==0 or showOrNot==2:
        continue
      
      y1, x1, y2, x2 = int(np.floor(bbox[0])), int(np.floor(bbox[1])), int(np.ceil(bbox[2])), int(np.ceil(bbox[3]))
      img_w_mask = plot_mask(mask_color, alpha, raw_image, segmentations[anno_idx])
      crop_w_mask = img_w_mask[y1:y2, x1:x2, :]
      
      category_names = ['background']+[x.strip() for x in self.category_name_string.split(';')] 
      fig_size_h = min(max(5, int(len(category_names)/2.5)), 10)
      fig, axs = plt.subplots(
        1, 4, figsize=(fig_size_w, fig_size_h),
        gridspec_kw={'width_ratios': [3, 1, 1, 2]}, 
        constrained_layout=True
      )
      #@ Draw bounding box.
      rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=line_thickness, edgecolor='r', facecolor='none')
      axs[0].add_patch(rect)
      axs[0].set_xticks([])
      axs[0].set_yticks([])
      axs[0].set_title(f'bbox: {y1, x1, y2, x2}   area: {(y2 - y1) * (x2 - x1)}   rpn score: {rpn_score:.4f}')
      axs[0].imshow(raw_image)
      #@ Draw image in a cropped region.
      crop = np.copy(raw_image[y1:y2, x1:x2, :])
      axs[1].set_xticks([])
      axs[1].set_yticks([])
      axs[1].set_title(f'predicted: {category_names[np.argmax(scores)]}')
      showLabel = category_names[np.argmax(scores)]
      axs[1].imshow(crop)
      #@ Draw segmentation inside a cropped region.
      axs[2].set_xticks([])
      axs[2].set_yticks([])
      axs[2].set_title('mask')
      axs[2].imshow(crop_w_mask)
      #@ Draw category scores.
      fontsize = max(min(fig_size_h/float(len(category_names))*45, 20), 8)
      for cat_idx in range(len(category_names)):
        axs[3].barh(cat_idx, scores[cat_idx], 
                    color='orange' if scores[cat_idx] == max(scores) else 'blue')
      axs[3].invert_yaxis()
      axs[3].set_axisbelow(True)
      axs[3].set_xlim(0, 1)
      plt.xlabel("confidence score")
      axs[3].set_yticks(range(len(category_names)))
      axs[3].set_yticklabels(category_names, fontdict={'fontsize': fontsize})

      f = plt.gcf()
      f.savefig('result/result_{id}.jpg'.format(id=anno_idx), bbox_inches="tight")
      f.clear()
      cnt += 1
    self.msg_label = QMessageBox(self)
    self.msg_label.setText("RESULTS SAVED!")
    self.msg_label.setInformativeText("Permitted Detection Counts: " + str(cnt) + '!')
    self.msg_label.setIcon(QMessageBox.Information)
    self.msg_label.show()
  
class originalLabel(QLabel):
  def __init__(self, parent=None):
    super(originalLabel, self).__init__((parent))
    self.parentWindow = parent
    self.setGeometry(QRect(0, 0, self.parentWindow.imageWidth, self.parentWindow.imageHeight))
    
    bytesPerLine = 3 * self.parentWindow.imageWidth
    cv2.cvtColor(self.parentWindow.img, cv2.COLOR_BGR2RGB, self.parentWindow.img)
    QImg = QImage(self.parentWindow.img.data, self.parentWindow.imageWidth, self.parentWindow.imageHeight, bytesPerLine, QImage.Format_RGB888)
    pixmap = QPixmap.fromImage(QImg)
    self.setPixmap(pixmap)
    
class customInputLabel(QLineEdit):
  def __init__(self, parent=None):
    super(customInputLabel, self).__init__((parent))
    self.parentWindow = parent
    
    re = QRegExp('[a-zA-Z,]+$')
    re_validato = QRegExpValidator(re)
    self.setValidator(re_validato)
    self.move(self.parentWindow.lb_0.x()+50, self.parentWindow.lb_0.y()+20)
    self.resize(510, 20)
    self.setText("Please input your custom label(s) to Zero-shot Detect! [separated by',']")
    self.textChanged.connect(self.parentWindow.textchange) 
    
class bboxInputLabel(QLineEdit):
  def __init__(self, parent=None):
    super(bboxInputLabel, self).__init__((parent))
    self.parentWindow = parent
    re = QRegExp('[0-9]+$')
    re_validato = QRegExpValidator(re)
    self.setValidator(re_validato)
    self.move(self.parentWindow.lb_0.x()+50, self.parentWindow.lb_0.y()+45)
    self.resize(160, 20)
    self.setText("Max bbox numbers:")
    self.textChanged.connect(self.parentWindow.numchange)   
    
    
class resultWindow(QMainWindow):
  def __init__(self, parent=None):
    super(resultWindow, self).__init__(parent)
    self.parentWindow = parent
    self.setWindowTitle('Detected objects. Zero-shot ends.')
    img = cv2.imread("tmp/ui.jpg")
    self.imageHeight, self.imageWidth, self.bytesPerComponent = img.shape
    self.resize(self.imageWidth, self.imageHeight)
    self.move(int((QDesktopWidget().screenGeometry().width()-self.width())/2),
              int((QDesktopWidget().screenGeometry().height()-self.height())/2))
        
    self.lb = resultLabel(self)
    self.lb.setGeometry(QRect(0, 0, self.imageWidth, self.imageHeight))
    bytesPerLine = 3 * self.imageWidth
    cv2.cvtColor(img, cv2.COLOR_BGR2RGB, img)
    QImg = QImage(img.data, self.imageWidth, self.imageHeight, bytesPerLine, QImage.Format_RGB888)
    pixmap = QPixmap.fromImage(QImg)
    self.lb.setPixmap(pixmap)
    self.lb.setCursor(Qt.CrossCursor)

class resultLabel(QLabel):
  def __init__(self, parent=None):
    super(resultLabel, self).__init__((parent))   
    self.parentWindow = parent
    self.rootWindow = self.parentWindow.parentWindow
    self.x0 = 0
    self.y0 = 0
    self.x1 = 0
    self.y1 = 0
    self.flag = False
    self.index = 0

  def mousePressEvent(self, event):
    self.flag = True
    self.x0 = event.x()
    self.y0 = event.y()
    #//print(self.x0, self.y0)
    self.index = 0
    ## TODO Area Cverlap ## 
    global indices_fg               ########
    global rescaled_detection_boxes ########
    global isSaved                  ########
    for self.index in indices_fg:
      #//print(self.index)
      ymin, xmin, ymax, xmax = rescaled_detection_boxes[self.index]
      #//print(ymin, xmin, ymax, xmax)
      if self.x0 < xmax and self.x0 > xmin and self.y0 < ymax and self.y0 > ymin and isSaved[self.index]!=3:
        self.cb = QComboBox(self)
        self.cb.move(self.x0, self.y0)
        self.cb.addItem('This bbox is selected')
        self.cb.addItem(QtGui.QIcon("./icon/icons8-assessments-90.png"),'Get its results')
        self.cb.addItem(QtGui.QIcon("./icon/icons8-multiplication-90.png"),'Discard it')
        self.cb.addItem(QtGui.QIcon("./icon/icons8-garbage-66.png"),'Discard it FOREVER')
        self.cb.setCurrentIndex(0)
        
        self.cb.currentIndexChanged[str].connect(self.oper)
        self.cb.show()
        break
      
  def oper(self, choice):
    global image                    ########
    global valid_indices            ########
    global max_boxes_to_draw        ########
    global detection_roi_scores     ########
    global category_indices         ########
    global segmentations            ########
    global min_rpn_score_thresh     ########
    #//print(i)
    if choice == "Get its results":
      #//print(self.index)
      # Global Variable
      isSaved[self.index] = 1
      self.mgb = QMessageBox(self)
      self.mgb.setIcon(QMessageBox.Information)
      self.mgb.setText("You have saved the detection result.")
      self.mgb.show()
      
    elif choice == "Discard it":
      #//print(self.index)
      # Global Variable
      isSaved[self.index] = 2
      self.mgb = QMessageBox(self)
      self.mgb.setIcon(QMessageBox.Warning)
      self.mgb.setText("You would ignore this bbox!")
      self.mgb.setInformativeText("Later, it will not be saved")
      self.mgb.show()
    
    elif choice == "Discard it FOREVER":
      isSaved[self.index] = 3
      self.mgb = QMessageBox(self)
      self.mgb.setIcon(QMessageBox.Warning)
      self.mgb.setText("Discard FOREVER")
      self.mgb.setInformativeText("CANNOT RECOVER!")
      self.mgb.show()
      
    self.cb.deleteLater()
    # Update Results
    image_with_detections = visualize_boxes_and_labels_on_image_array( \
      np.array(image),
      rescaled_detection_boxes[indices_fg],
      valid_indices[:max_boxes_to_draw][indices_fg],
      detection_roi_scores[indices_fg],   
      category_indices, #numbered_category_indices,
      user=isSaved[indices_fg], 
      instance_masks=segmentations[indices_fg],
      use_normalized_coordinates=False,
      max_boxes_to_draw=max_boxes_to_draw, min_score_thresh=min_rpn_score_thresh,
      skip_scores=False, skip_labels=False,
    )
    
    img = cv2.imread("tmp/ui.jpg")
    height, width, bytesPerComponent = img.shape
    self.setGeometry(QRect(0, 0, width, height))
    bytesPerLine = 3 * width
    cv2.cvtColor(img, cv2.COLOR_BGR2RGB, img)
    QImg = QImage(img.data, width, height, bytesPerLine, QImage.Format_RGB888)
    pixmap = QPixmap.fromImage(QImg)
    self.setPixmap(pixmap)
    self.setCursor(Qt.CrossCursor)
    
  def mouseReleaseEvent(self,event):
    self.flag = False


####### Main Function ######
def main():
  app = QApplication(sys.argv)
  win_1 = firstWindow()
  
  win_1.show()
  app.exec_()
main()

START UI


2022-07-18 21:39:51.076 python[14419:314224] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


person;car
START DETECTING!
Valid Indices Cnt:  254
Building text embeddings...


100%|██████████| 3/3 [00:08<00:00,  2.94s/it]


0
1
2
3
4
Update Detection Window
car;person
START DETECTING!
Valid Indices Cnt:  254
Building text embeddings...


...

0
1
2
3
4
Update Detection Window
car;person
START DETECTING!
Valid Indices Cnt:  254
Building text embeddings...


100%|██████████| 3/3 [00:08<00:00,  2.73s/it]


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
Update Detection Window
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
car
START DETECTING!
Valid Indices Cnt:  254
Building text embeddings...


100%|██████████| 2/2 [00:05<00:00,  2.70s/it]


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
Update Detection Window
